In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
#rozpakowywanie danych
data = pd.read_pickle("Data.pickle")

In [3]:
data

,School Name,Region,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,California Institute of Technology (CIT),California,Engineering,75500.0,123000.0,62650.0,104000.0,161000.0,175571.428571
1,Harvey Mudd College,California,Engineering,71800.0,122000.0,62650.0,96000.0,180000.0,175571.428571
2,"University of California, Berkeley",California,State,59900.0,112000.0,59500.0,81000.0,149000.0,201000.000000
3,Occidental College,California,Liberal Arts,51900.0,105000.0,NaN,54800.0,157000.0,NaN
4,Cal Poly San Luis Obispo,California,State,57200.0,101000.0,55000.0,74700.0,133000.0,178000.000000
...,...,...,...,...,...,...,...,...,...
255,Fitchburg State College,Northeastern,State,42400.0,72600.0,43300.0,56100.0,99600.0,151000.000000
256,University Of Maine,Northeastern,State,41200.0,72100.0,41700.0,55600.0,99300.0,141000.000000
257,State University of New York (SUNY) at Potsdam,Northeastern,State,38000.0,70300.0,35100.0,51200.0,100000.0,179000.000000
258,State University of New York (SUNY) at Fredonia,Northeastern,State,37800.0,66200.0,32800.0,44200.0,93300.0,181000.000000


## Analiza korelacji między medianą zarobków początkowych absolwentów a ich zarobków 10 lat od rozpoczęcia kariery

In [4]:
st.pearsonr(data['Starting Median Salary'],data['Mid-Career Median Salary'])

(0.9009217367567477, 1.6152279024528496e-95)

Współczynnik korelacji r-Pearsona między początkową medianą zarobków a medianą zarobków 10 lat od rozpoczęcia kariery jest bardzo wysoki. Wskazuje to na to, że istnieje bardzo wysoka korelacja między początkowymi zarobkami absolwentów danego uniwersytetu, a ich zarobkami w dalszym życiu zawodowym. Ponadto p o bardzo małej wartości (znacząco poniżej poziomu alfa=0,05) świadczy o wiarygodności korelacji.

## Test istotności różnic między medianą zarobków 10 lat od rozpoczęcia kariery absolwentów uniwersytetów z regionu Southern i regionu Northeast

In [5]:
# Zbiory zostały wybrane ze względu na to, że są to regiony o największej liczbie uniwersytetów
# oraz są prawie równoliczne.
# Badanie liczebności zbiorów:
x=[]
y=[]
for region in data.Region.unique():
    x.append(len(data.loc[data['Region'] == region].index))
    y.append(region)
print (x, '\n', y)

[26, 36, 62, 67, 69] 
 ['California', 'Western', 'Midwestern', 'Southern', 'Northeastern']


Najbardziej licznymi zbiorami są Southern oraz Northeastern, mają również zbliżoną liczność. W celu wykonania testu T z większego zbioru została pobrana próbka o liczności mniejszego zbioru. Pominięcie jedynie dwóch rekordów nie powinno znacząco wpłynąć na analizę.

In [6]:
# Pobranie losowej próby z większego zbioru
sample=data.loc[data['Region'] == 'Northeastern'].sample(n=67)
# dla wygody stworzenie zbioru z próbą z całości mniejszego zbioru
sample2=data.loc[data['Region'] == 'Southern']

Test T wykonywany w poniższy sposób wymaga spełnienia założenia o zbliżonym do normalnego rozkładnie danych. W związku z tym wykonany został test dla rozkładu normalnego dla obu zbiorów. Przy założonym poziomie istotności alfa = 0,05 wynik p mniejszy od alfa oznacza odrzucenie hipotezy zerowej o normalności rozkładu. Wynik p większy od alfa - hipoteza o normalności rozkładu nie jest odrzucana.

In [7]:
st.normaltest(sample['Mid-Career Median Salary'])

NormaltestResult(statistic=3.729395454042106, pvalue=0.15494303790076197)

In [8]:
st.normaltest(sample2['Mid-Career Median Salary'])

NormaltestResult(statistic=1.7663357498922758, pvalue=0.41347101035437683)

Oba rozkłady można uznać za normalne. Następnie należy sprawdzić równość wariancji. W tym celu wykonany został test levene'a. Podobnie jak w teście rozkładu, p mniejsze od alfa świadczy o tym, że hipotezę zerową o równości wariacji należy odrzucić.

In [9]:
st.levene(sample['Mid-Career Median Salary'],sample2['Mid-Career Median Salary'])

LeveneResult(statistic=14.97141197800633, pvalue=0.0001707427883300336)

Wariancje nie są równe, hipotezę odrzucamy, w związku z tym wykonując test istotności różnic wykorzystujemy dodatkowy parametr informujący o nierównych wariancjach. P niższe od alfa będzie świadczyło o tym, że róznice są istotne

In [10]:
st.ttest_ind(sample['Mid-Career Median Salary'],sample2['Mid-Career Median Salary'],equal_var= False)

Ttest_indResult(statistic=5.492292555377446, pvalue=2.4957269581564933e-07)

Wynikowe p jest zdecydowanie poniżej poziomu alfa. Test istotności wykazał więc, że różnice w medianie zarobków 10 lat po rozpoczęciu kariery dla wybranych regionów są statystycznie istotne.